In [ ]:
#pip install websocket-client

In [ ]:
#pip install python-binance

In [ ]:
# installed TA-Lib through Anaconda (conda install -c conda-forge ta-lib)

In [ ]:
import time
import websocket
import rel
import _thread

from binance.client import Client

import json

import numpy as np
import pandas as pd

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_key = file.readlines()[0].split("\n")[0] # to read public key from text file

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_secret = file.readlines()[1].split("\n")[0] # to read secret key from text file

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
symbol = "ETHUSDT" #in uppercase
frequency = "1m" #in lowercase except month
endpoint = f"wss://stream.binance.com:9443/ws/{symbol}@kline_{frequency}"

In [ ]:
#parameters for obtaining historical data

klines_dict = {
    
    "1m" : Client.KLINE_INTERVAL_1MINUTE,
    "3m" : Client.KLINE_INTERVAL_3MINUTE,
    "5m" : Client.KLINE_INTERVAL_5MINUTE,
    "15m" : Client.KLINE_INTERVAL_15MINUTE,
    "30m" : Client.KLINE_INTERVAL_30MINUTE,
    "1h" : Client.KLINE_INTERVAL_1HOUR,
    "2h" : Client.KLINE_INTERVAL_2HOUR,
    "4h" : Client.KLINE_INTERVAL_4HOUR,
    "6h" : Client.KLINE_INTERVAL_6HOUR,
    "8h" : Client.KLINE_INTERVAL_8HOUR,
    "12h" : Client.KLINE_INTERVAL_12HOUR,
    "1d" : Client.KLINE_INTERVAL_1DAY,
    "3d" : Client.KLINE_INTERVAL_3DAY,
    "1w" : Client.KLINE_INTERVAL_1WEEK,
    "1M" : Client.KLINE_INTERVAL_1MONTH

}

#how far back you want historical data? examples:

# 30 minutes ago UTC
# 1 day ago UTC
# 1 Dec, 2017

In [ ]:
klines = client.get_historical_klines(symbol, klines_dict[frequency], "30 day ago UTC")

In [ ]:
open_time, open_, high, low, close, volume, close_time, volume_quoted, num_trades = ([] for i in range(9))

In [ ]:
for candle in klines:
    open_time.append(candle[0])
    open_.append(candle[1])
    high.append(candle[2])
    low.append(candle[3])
    close.append(candle[4])
    volume.append(candle[5])
    close_time.append(candle[6])
    volume_quoted.append(candle[7])
    num_trades.append(candle[8])

open_time = np.array(open_time).astype(float)
open_ = np.array(open_).astype(float)
high = np.array(high).astype(float)
low = np.array(low).astype(float)
close = np.array(close).astype(float)
volume = np.array(volume).astype(float)
close_time = np.array(close_time).astype(float)
volume_quoted = np.array(volume_quoted).astype(float)
num_trades = np.array(num_trades).astype(float)

In [ ]:
def on_open(ws):
    print("Connection established")
    print("")

def on_message(ws, message):
    
    global open_time, open_, high, low, close, volume, close_time, volume_quoted, num_trades
    
    j_message = json.loads(message)
    candle = j_message["k"]
    
    is_candle_closed = candle["x"]
    
    open_time_new = float(candle["t"])
    close_time_new = float(candle["T"])
    open_new = float(candle["o"])
    close_new = float(candle["c"])
    high_new = float(candle["h"])
    low_new = float(candle["l"])
    volume_new = float(candle["v"])
    num_trades_new = float(candle["n"])
    volume_quoted_new = float(candle["q"])

    
    if is_candle_closed:
                
        open_time_new = np.array(open_time_new)
        open_new = np.array(open_new)
        high_new = np.array(high_new)
        low_new = np.array(low_new)
        close_new = np.array(close_new)
        volume_new = np.array(volume_new)
        close_time_new = np.array(close_time_new)
        volume_quoted_new = np.array(volume_quoted_new)
        num_trades_new = np.array(num_trades_new)
        
        open_time = np.insert(open_time, len(open_time), open_time_new, axis=0)
        open_ = np.insert(open_, len(open_), open_new, axis=0)
        high = np.insert(high, len(high), high_new, axis=0)
        low = np.insert(low, len(low), low_new, axis=0)
        close = np.insert(close, len(close), close_new, axis=0)
        volume = np.insert(volume, len(volume), volume_new, axis=0)
        close_time = np.insert(close_time, len(close_time), close_time_new, axis=0)
        volume_quoted = np.insert(volume_quoted, len(volume_quoted), volume_quoted_new, axis=0)
        num_trades = np.insert(num_trades, len(num_trades), num_trades_new, axis=0)

        print(f"Candle number {len(close)} added at closing price of {close[-1]}")
    
def on_error(ws, error):
    print(error)
    
def on_close(ws, close_status_code, close_msg):
    print("Connection closed")

In [ ]:
ws = websocket.WebSocketApp(endpoint,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

In [ ]:
ws.run_forever() #numpy arrays will continue to add live information